In [1]:
#install the following libraries if required
#use commands :
# pip install numpy
# pip install opencv-python
import numpy as np
import cv2 as cv


def detect_cloak(frame, bg):
    #convert image to HSV
    hsv_image = cv.cvtColor(frame, cv.COLOR_RGB2HSV)
    #create a mask and segment frame based on choice of H_val
    sensitivity = 20
    H_val = 20 #change this value to use a different colour
    light_blue = np.array([H_val - sensitivity,60,60])
    dark_blue = np.array([H_val + sensitivity,255,255])
    #create segmentation mask
    mask = cv.inRange(hsv_image, light_blue, dark_blue)
    
    #use morphology to close unwanted gaps 
    kernel_size = 10 #bigger kernel_size, lesser gaps
    kernel = np.ones((kernel_size,kernel_size), np.uint8)
    #closing = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel) 
    closing = cv.dilate(mask, kernel, iterations=1) #works better in this case as it gives a more rounded, smoother border to segment
    
    #find the contour coordinates, sort them to get the biggest area and create a mask of that area
    contours,_ = cv.findContours(closing, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contour_sort = sorted(contours,key=cv.contourArea,reverse=True)[:5]
    contour_mask = cv.fillPoly(np.zeros((500,500,3), dtype=np.uint8), pts=[contour_sort[0]], color=(255,255,255))
    
    #we create 2 masks
    #object_mask is masking the current frame on the desired area with a black polygon
    object_mask = cv.fillPoly(frame,pts=[contour_sort[0]], color=(0,0,0))
    #bg_mask creates an image where only the segmented part is filled with the corresponding part from bg and the rest is black
    bg_mask = np.bitwise_and(contour_mask,bg) #bitwise_and returns black when black & anything else... anything else when white & anything else
    
    #now we finally make the image by superimposing our 2 masks on each other using bitwise_or
    final = cv.bitwise_or(bg_mask,object_mask)
    
    return final;
    
    
    
#create video capture object
cap = cv.VideoCapture(0) 

#read frame from object to store as background for our program 
ret, bg = cap.read()
bg = cv.resize(bg,(500,500))
#show the bg image
cv.imshow('Frame',bg)
#wait till user presses any key then destroy th window 
cv.waitKey(0)
cv.destroyAllWindows()

#this part of the code is used to save each frame in .mp4 file created in the same directory this code exists in
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('output.mp4', fourcc, 30.0, (500,500))

#now keep reading frames using a loop
while(True):
    ret,frame = cap.read()
    frame = cv.resize(frame, (500, 500))
    #pass the current frame and the frame we used as our background to create the mask
    image = detect_cloak(frame, bg)
    
    out.write(image) #saving each frame locally
    
    #display the processed frame
    cv.imshow('Image',image)
    
    #set exit conditions to close the application
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

#release all the objects that were created and destroy windows created 
cap.release() 
out.release()
cv.destroyAllWindows()